# Final Exam Assignment (40 points - Total of 45 is Possible)
## Due December 7, 2022, @ 8:00 am
Note that there will be no extensions given for this assignment as there is a tight timeline for grading. 

For this assignment, I have provided each of you with your own training dataset. Your goal is to train a deep neural network to uncover the code image provided to you. 

I will provide you with instructions throughout. 

In [1]:
# Add your import statements here
import os
import numpy as np
import urllib
import time
import sys
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import DataLoader
from torch import nn

In [2]:
# This is a tool I have provided you to help you download your file.

def download_file(url, filename):
    """
    A function that downloads the data file from a URL
    Parameters
    ----------
    url : string
        url where the file to download is located
    filename : string
        location where to save the file
    reporthook : function
        callback to display the download progress
    """
    if not os.path.isfile(filename):
        urllib.request.urlretrieve(url, filename, reporthook)
        
def reporthook(count, block_size, total_size):
    """
    A function that displays the status and speed of the download
    """

    global start_time
    if count == 0:
        start_time = time.time()
        return
    duration = time.time() - start_time
    progress_size = int(count * block_size)
    speed = int(progress_size / (1024 * duration + 0.0001))
    percent = int(count * block_size * 100 / total_size)
    sys.stdout.write("\r...%d%%, %d MB, %d KB/s, %d seconds passed" %
                     (percent, progress_size / (1024 * 1024), speed, duration))
    sys.stdout.flush()

In [3]:
# You can download your file by typing your first name into the name block
# The name used is the first part of your first name as listed in BB learn
# If you have problems downloading the data please reach out to me

name = 'Lauren'
download_file(f'https://zenodo.org/record/7339649/files/data_{name}.npz?download=1','data.npz')

## Loading the Data (3 points)
The data is provided to you as a compressed NumPy array saved as 'data.npz'. When working with real data you might need to figure out how data is stored. Use the information on 'npz' files to figure out what data you have. The data file contains three NumPy arrays. 
1. The features for the training dataset
2. The regression values for the training dataset
3. The validation features that contain your code

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
#mount drive, change directory to access file
!cd "/content/drive/MyDrive/t680"

In [6]:
# Your Code goes here
#os.listdir(".")
#data = np.load('data.npz', allow_pickle = True)
#lst = data.files
#data = load_npz("data.npz")
#with np.load('data.npz', allow_pickle=True) as data:
#    training_feat = data['training_feat']

with np.load('data.npz', allow_pickle=True) as data:
    training_feat = data['training_feat']   #features for training dataset
    training_true = data['training_true']   #regression values for training dataset
    validation_feat = data['validation_feat'] #validation features that contain your code
     

## Preprocessing the Data (5 points)

You should explore the data and figure out the best way to preprocess the data. 

Hints: 
1. For the regression values, these at the end will represent colors in RGB space from [0,1]. It is recommended to use a max-min scalar between 0 and 1. 
2. For the training features, you should look at the data and determine the best scaling method. Look at our class notes for a reminder of what other scaler might be useful. 

In [7]:
# Your code goes here
#1. For regression values, use max-min scalar between 0 and 1
mm_scaler = MinMaxScaler()
scaled_training_true = mm_scaler.fit_transform(training_true)   

#2. For training features, use standard scaler
s_scaler =  StandardScaler()
scaled_training_feat = s_scaler.fit_transform(training_feat)

## Building the Dataset (5 points)

When training neural networks it is important to build a dataset that allows the machinery to sample the data. This also can be used to conduct some preprocessing of the data to make it work with PyTorch. 

I have provided you with the framework for a Dataset Class. 

You should:
1. Convert the x and y data to a tensor 'float32' and put it on the GPU.
2. Save the len of the data
3. Add the code so when `__getitem__` is called it returns the x and y values
3. make it so `__len__` returns the lenght when calle


In [8]:
class Data(): #TODO: what is Dataset inheritance in the copied code
  '''Dataset Class to store the samples and their corresponding labels, 
  and DataLoader wraps an iterable around the Dataset to enable easy access to the samples.
  '''

  def __init__(self, X: np.ndarray, y: np.ndarray, device = 'cuda') -> None:

    # need to convert float64 to float32 else 
    # will get the following error
    # RuntimeError: expected scalar type Double but found Float
    self.X = X.astype(np.float32)
    self.y = y.astype(np.float32)
    self.len = len(self.X)
  
  def __getitem__(self, index: int) -> tuple:
    return self.X[index], self.y[index]
    #TODO: why is the index needed here? not explained in the problem statement

  def __len__(self) -> int:
    return self.len

## Train-test Split (3 points)

1. You should conduct a train-test split of the training data so you can make sure that your model does not overfit the data. A good ratio is 66/33 train 
2. You should instantiate the training dataset using the data class implemented above.


In [9]:
X_train, X_test, y_train, y_test = train_test_split(scaled_training_feat, scaled_training_true, test_size = 0.33, random_state = 42)
training_data = Data(X_train, y_train)

## Build the Dataloader (3 points)

Pytorch uses DataLoaders to efficiently sample from a training dataset. Instantiate a Pytorch DataLoader using the dataset. 

You should set the following parameters:
1. Batch size = 64
2. Shuffle = True

In [10]:
train_dataloader = DataLoader(training_data, batch_size = 64, shuffle = True)

## Building a Neural Network (5 points)

Using the provided class framework which inherits the `nn.Module` type in PyTorch builds a 4-layer neural network to complete the multiple regression.



In [11]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

class Neural_Network(nn.Module):
  ''' Regression Model
  ''' 

  # note, you can ignore the `:int` and `-> None` this is just more advanced doctring syntax
  def __init__(self, input_dim: int, hidden_dim: int, output_dim: int) -> None:
      '''The network has 4 layers
            - input layer
            - ReLu
            - hidden layer
            - ReLu
            - hidden layer
            - ReLu
            - output layer
      '''
      super(Neural_Network, self).__init__()
      # in this part you should intantiate each of the layer components
      self.flatten = nn.Flatten()
      self.linear_relu_stack = nn.Sequential(
          nn.Linear(input_dim, input_dim),  #input
          nn.ReLU(),                        #ReLu
          nn.Linear(input_dim, hidden_dim), #hidden
          nn.ReLU(),                        #ReLu
          nn.Linear(hidden_dim, output_dim),#hidden
          nn.ReLU(),                        #ReLu
          nn.Linear(output_dim, output_dim))#output

  def forward(self, x: torch.Tensor) -> torch.Tensor:
      # In this part you should build a model that returns the 3 outputs of the regression
      # Type your code here
      x = self.flatten(x)
      x = self.linear_relu_stack(x)
      #TODO: double check, class code uses a different variable to be returned
      
      return x
  

Using cuda device


## Instantiate the Model (3 points)



In [12]:
# number of features (len of X cols)
input_dim = X_train.shape[1]
# number of hidden layers set this to 50
hidden_layers = 5
# Add the number of output dimensions
output_dim = 3

print(input_dim, hidden_layers, output_dim)

30 5 3


In [13]:
# initiate the regression model
# make sure to put it on your GPU
model = Neural_Network(input_dim, hidden_layers, output_dim).to(device)
print(model)

# criterion to computes the loss between input and target
# Choose a good criteria
loss_fn = nn.MSELoss() #for regression tasks
# optimizer that will be used to update weights and biases
optimizer = torch.optim.Adam(model.parameters(), lr = 3e-5)
# you can choose any optimizer. I would recommend ADAM.
# This problem should not be hard to optimize. A good starting learning rate is 3e-5. 

Neural_Network(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=30, out_features=30, bias=True)
    (1): ReLU()
    (2): Linear(in_features=30, out_features=5, bias=True)
    (3): ReLU()
    (4): Linear(in_features=5, out_features=3, bias=True)
    (5): ReLU()
    (6): Linear(in_features=3, out_features=3, bias=True)
  )
)


## Train the Model (5 points)

Training the model is conducted in a number of steps using loops.

1. Set up a loop for each epoch
2. Set a parameter to save the running loss
3. Set up a nested loop that goes through the batches from the DataLoader you built
    - I would recommend using enumerate to include the counts in the loop
    - The dataloader will return a tuple that is the inputs and the labels
4. Conduct the forward propagation of the model
    - Give the model the inputs and compute the outputs
    - Compute the loss given the criteria. 
5. Use the zero gradient method to remove the gradients from the optimizer
6. Use the backward method to compute the gradients
7. Use the step method in the optimizer to take an optimization step
8. Compute the running loss by calling the item method and adding it to the running loss for each minibatch
9. For each epoch print the epoch and the loss

Note: If you find this challenging I would recommend that you look at examples of other pytorch training loops online. This is a very standard workflow. 

In [14]:
# start training
epochs = 20 # sets the number of epochs to train 20 should be sufficent.
# This should take about 5-10 minutes to train.

# Your code should go here 
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X.cuda())
        loss = loss_fn(pred, y.cuda())

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    #test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 0.154061  [    0/67000]
loss: 0.161866  [ 6400/67000]
loss: 0.169725  [12800/67000]
loss: 0.151751  [19200/67000]
loss: 0.156710  [25600/67000]
loss: 0.142009  [32000/67000]
loss: 0.146157  [38400/67000]
loss: 0.141429  [44800/67000]
loss: 0.135038  [51200/67000]
loss: 0.144667  [57600/67000]
loss: 0.137341  [64000/67000]
Epoch 2
-------------------------------
loss: 0.152244  [    0/67000]
loss: 0.145796  [ 6400/67000]
loss: 0.138221  [12800/67000]
loss: 0.132646  [19200/67000]
loss: 0.145027  [25600/67000]
loss: 0.130101  [32000/67000]
loss: 0.132970  [38400/67000]
loss: 0.135087  [44800/67000]
loss: 0.132133  [51200/67000]
loss: 0.124323  [57600/67000]
loss: 0.122313  [64000/67000]
Epoch 3
-------------------------------
loss: 0.115280  [    0/67000]
loss: 0.124893  [ 6400/67000]
loss: 0.123447  [12800/67000]
loss: 0.130655  [19200/67000]
loss: 0.110241  [25600/67000]
loss: 0.117155  [32000/67000]
loss: 0.123664  [38400/67000]
loss: 0.09

## Validate the Model (3 points)

Use the test dataset from the train-test split to make sure your model is not overfitting

1. You can build a dataloader as you did before, this time with the test data.
2. Build a validation loop, which you should use `with torch.no_grad()` to make sure you do not modify the gradients, or weights. This will fix your model. 
3. Instantiate the loss to be 0.
4. Build a similar loop to grab the validation dataset. 
5. Compute the predictions with the model.
6. Compute the loss using your loss criteria.
7. Print the final loss determined.

In [16]:
test_data = Data(X_test, y_test)
test_dataloader = DataLoader(test_data, batch_size = 64, shuffle = True)

def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss = 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X.cuda())
            test_loss += loss_fn(pred, y.cuda()).item()
            

    test_loss /= num_batches
    print(test_loss)
    
    
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    test_loop(test_dataloader, model, loss_fn)
print("Done!")    

Epoch 1
-------------------------------
0.015340705456342115
Epoch 2
-------------------------------
0.015339477482259042
Epoch 3
-------------------------------
0.015339783519188794
Epoch 4
-------------------------------
0.015340931673873534
Epoch 5
-------------------------------
0.015341206767791233
Epoch 6
-------------------------------
0.015338117901105867
Epoch 7
-------------------------------
0.015342100012628722
Epoch 8
-------------------------------
0.01534041668136799
Epoch 9
-------------------------------
0.015339488744764596
Epoch 10
-------------------------------
0.015339770868723823
Epoch 11
-------------------------------
0.015338295298202555
Epoch 12
-------------------------------
0.015340391777513562
Epoch 13
-------------------------------
0.015341373167701008
Epoch 14
-------------------------------
0.015342370673739749
Epoch 15
-------------------------------
0.01533929541591526
Epoch 16
-------------------------------
0.015344009875471508
Epoch 17
----------

<p style="color:blue"> Question: Is your model overfitting or not? How do you know? (3 points) </p>

The loss during the testing loop seems to be slightly decreasing with each epoch, and is resting at about 0.0153. This is relatively close to the losses seen in the training regimine. Therefore, I do not think the model is currently overfitting.

## Crack Your Code (3 points)

1. You can build a dataloader as you did before, this time with the validation features to view your code.
2. Build a loop, you should use `with torch.no_grad()` to make sure you do not modify the gradients or weights. This will fix your model. 
3. Compute the predictions of your model. 
    - Make sure you do all the same preprocess, the data has the same datatype, and is on the same device

In [ ]:
# Your code goes here

## Reveal Your Code (3 points)

Your code is an image. there are (65536, 3) predictions this corresponds to a (256,256,3) RGB image. 
1. Use the detach() method to remove the gradients from the tensor
2. Transfer the tensor back to the 'cpu' if you had it on a GPU
3. Reshape the image into a 256,256,3 array. 
4. Plot your successful result.

In [ ]:
# Your Code goes here